In [1]:
%matplotlib inline


Functional connectivity matrices for group analysis of connectomes
==================================================================

This example compares different kinds of functional connectivity between
regions of interest : correlation, partial correlation, as well as a kind
called **tangent**. The resulting connectivity coefficients are used to
discriminate ADHD patients from healthy controls and the **tangent kind**
**outperforms** the standard connectivity kinds.



In [2]:
# Matrix plotting from Nilearn: nilearn.plotting.plot_matrix
import numpy as np
import matplotlib.pylab as plt


def plot_matrices(matrices, matrix_kind):
    n_matrices = len(matrices)
    fig = plt.figure(figsize=(n_matrices * 4, 4))
    for n_subject, matrix in enumerate(matrices):
        plt.subplot(1, n_matrices, n_subject + 1)
        matrix = matrix.copy()  # avoid side effects
        # Set diagonal to zero, for better visualization
        np.fill_diagonal(matrix, 0)
        vmax = np.max(np.abs(matrix))
        title = '{0}, subject {1}'.format(matrix_kind, n_subject)
        plotting.plot_matrix(matrix, vmin=-vmax, vmax=vmax, cmap='RdBu_r',
                             title=title, figure=fig, colorbar=False)

Load ADHD dataset and MSDL atlas
--------------------------------
We study only 20 subjects from the ADHD dataset, to save computation time.



In [3]:
from nilearn import datasets

adhd_data = datasets.fetch_adhd(n_subjects=20)

/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)



Dataset created in /Users/nikkibytes/nilearn_data/adhd



Downloaded 3072 of 3072 bytes (100.0%,    0.0s remaining) ...done. (1 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/74a52d3552145e11365bb68453d1a70c/adhd40_metadata.tgz..... done.
/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/nilearn/datasets/func.py:502: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype=None)


Downloaded 44414948 of 44414948 bytes (100.0%,    0.0s remaining) ...done. (6 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_0010042.tgz..... done.


Downloaded 45583539 of 45583539 bytes (100.0%,    0.0s remaining) ...done. (8 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_0010064.tgz..... done.


...

Downloaded 60663491 of 60663491 bytes (100.0%,    0.0s remaining) ...done. (13 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_1562298.tgz..... done.


Downloaded 20628764 of 20628764 bytes (100.0%,    0.0s remaining) ...done. (5 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_1679142.tgz..... done.


Downloaded 33119052 of 33119052 bytes (100.0%,    0.0s remaining) ...done. (6 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_2014113.tgz..... done.


Downloaded 45455025 of 45455025 bytes (100.0%,    0.0s remaining) ...done. (8 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_2497695.tgz..... done.


Downloaded 57992968 of 57992968 bytes (100.0%,    0.0s remaining) ...done. (8 seconds, 0 min)
Extracting data from /Users/nikkibytes/nilearn_data/adhd/51ca93ac61e627213eb5a3bb40abfcc0/adhd40_2950754.tgz..... done.


We use probabilistic regions of interest (ROIs) from the MSDL atlas.



In [17]:
msdl_data = datasets.fetch_atlas_msdl()
msdl_coords = msdl_data.region_coords
n_regions = len(msdl_coords)
#print('MSDL has {0} ROIs, part of the following networks :\n{1}.'.format(
#    n_regions, msdl_data.networks))


/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/numpy/lib/npyio.py:2322: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


Region signals extraction
-------------------------
To extract regions time series, we instantiate a
:class:`nilearn.input_data.NiftiMapsMasker` object and pass the atlas the
file name to it, as well as filtering band-width and detrending option.



In [18]:
from nilearn import input_data

masker = input_data.NiftiMapsMasker(
    msdl_data.maps, resampling_target="data", t_r=2.5, detrend=True,
    low_pass=.1, high_pass=.01, memory='nilearn_cache', memory_level=1)

Then we compute region signals and extract useful phenotypic informations.



In [21]:
adhd_subjects = []
pooled_subjects = []
site_names = []
adhd_labels = []  # 1 if ADHD, 0 if control
for func_file, confound_file, phenotypic in zip(
        adhd_data.func, adhd_data.confounds, adhd_data.phenotypic):
    time_series = masker.fit_transform(func_file, confounds=confound_file)
    pooled_subjects.append(time_series)
    is_adhd = phenotypic['adhd']
    if is_adhd:
        adhd_subjects.append(time_series)

    site_names.append(phenotypic['site'])
    adhd_labels.append(is_adhd)

print('Data has {0} ADHD subjects.'.format(len(adhd_subjects)))


Data has 13 ADHD subjects.


In [22]:
pooled_subjects

[array([[  -0.58765541,   -1.05472835,    3.03562641, ...,   -1.58074974,
           -0.69711819,    1.78376895],
        [   9.92830447,   23.50980548,  165.0579614 , ...,  -33.78418624,
          -61.63485764,  -43.79106255],
        [   9.75531607,   48.76169616,  230.87467278, ...,  -39.99128819,
         -118.28597347,  -66.08393488],
        ...,
        [  -7.11366497,  -40.16357524,  -18.69711025, ...,   64.41141801,
          -73.57126115,  131.77627614],
        [  11.12994871,  -37.8814891 ,  -20.49034909, ...,   73.48539761,
          -78.30014021,  104.89936242],
        [   6.74716922,    7.65476576,    5.19421998, ...,    3.9971927 ,
            7.1628443 ,   -4.49433167]]),
 array([[  -7.93123261,   -7.35334281,   11.58426564, ...,    7.15193038,
           -3.57387755,   -0.53844873],
        [ 163.73601781,   21.44515039,   65.70093812, ...,  152.01382408,
           63.80522043,   48.52860194],
        [ 179.44455788,   62.01088155,   29.4314462 , ...,  194.50699803,

ROI-to-ROI correlations of ADHD patients
----------------------------------------
The simpler and most commonly used kind of connectivity is correlation. It
models the full (marginal) connectivity between pairwise ROIs. We can
estimate it using :class:`nilearn.connectome.ConnectivityMeasure`.



In [ ]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(kind='correlation')

From the list of ROIs time-series for ADHD subjects, the
`correlation_measure` computes individual correlation matrices.



In [ ]:
correlation_matrices = correlation_measure.fit_transform(adhd_subjects)

# All individual coefficients are stacked in a unique 2D matrix.
print('Correlations of ADHD patients are stacked in an array of shape {0}'
      .format(correlation_matrices.shape))

as well as the average correlation across all fitted subjects.



In [ ]:
mean_correlation_matrix = correlation_measure.mean_
print('Mean correlation has shape {0}.'.format(mean_correlation_matrix.shape))

We display the connectomes of the first 3 ADHD subjects and the mean
correlation matrix over all ADHD patients.



In [ ]:
from nilearn import plotting

plot_matrices(correlation_matrices[:4], 'correlation')
plotting.plot_connectome(mean_correlation_matrix, msdl_coords,
                         title='mean correlation over 13 ADHD subjects')

Look at blocks structure, reflecting functional networks.



Examine partial correlations
----------------------------
We can also study **direct connections**, revealed by partial correlation
coefficients. We just change the `ConnectivityMeasure` kind



In [ ]:
partial_correlation_measure = ConnectivityMeasure(kind='partial correlation')

and repeat the previous operation.



In [ ]:
partial_correlation_matrices = partial_correlation_measure.fit_transform(
    adhd_subjects)

Most of direct connections are weaker than full connections, resulting
in a sparse mean connectome graph.



In [ ]:
plot_matrices(partial_correlation_matrices[:4], 'partial')
plotting.plot_connectome(
    partial_correlation_measure.mean_, msdl_coords,
    title='mean partial correlation over 13 ADHD subjects')

Extract subjects variabilities around a robust group connectivity
-----------------------------------------------------------------
We can use **both** correlations and partial correlations to capture
reproducible connectivity patterns at the group-level and build a **robust**
**group connectivity matrix**. This is done by the **tangent** kind.



In [ ]:
tangent_measure = ConnectivityMeasure(kind='tangent')

We fit our ADHD group and get the group connectivity matrix stored as
in `tangent_measure.mean_`, and individual deviation matrices of each subject
from it.



In [ ]:
tangent_matrices = tangent_measure.fit_transform(adhd_subjects)

`tangent_matrices` model individual connectivities as
**perturbations** of the group connectivity matrix `tangent_measure.mean_`.
Keep in mind that these subjects-to-group variability matrices do not
straight reflect individual brain connections. For instance negative
coefficients can not be interpreted as anticorrelated regions.



In [ ]:
plot_matrices(tangent_matrices[:4], 'tangent variability')
plotting.plot_connectome(
    tangent_measure.mean_, msdl_coords,
    title='mean tangent connectivity over 13 ADHD subjects')

The mean connectome graph is not as sparse as partial correlations graph,
yet it is less dense than correlations graph.



What kind of connectivity is most powerful for classification?
--------------------------------------------------------------
*ConnectivityMeasure* can output the estimated subjects coefficients
as a 1D arrays through the parameter *vectorize*.



In [ ]:
connectivity_biomarkers = {}
kinds = ['correlation', 'partial correlation', 'tangent']
for kind in kinds:
    conn_measure = ConnectivityMeasure(kind=kind, vectorize=True)
    connectivity_biomarkers[kind] = conn_measure.fit_transform(pooled_subjects)

# For each kind, all individual coefficients are stacked in a unique 2D matrix.
print('{0} correlation biomarkers for each subject.'.format(
    connectivity_biomarkers['correlation'].shape[1]))

Note that we use the **pooled groups**. This is crucial for **tangent** kind,
to get the displacements from a **unique** `mean_` of all subjects.



We stratify the dataset into homogeneous classes according to phenotypic
and scan site. We then split the subjects into 3 folds with the same
proportion of each class as in the whole cohort



In [ ]:
from sklearn.model_selection import StratifiedKFold

classes = ['{0}{1}'.format(site_name, adhd_label)
           for site_name, adhd_label in zip(site_names, adhd_labels)]
cv = StratifiedKFold(n_splits=3)

and use the connectivity coefficients to classify ADHD patients vs controls.



In [ ]:
# Note that in cv.split(X, y),
# providing y is sufficient to generate the splits and
# hence np.zeros(n_samples) may be used as a placeholder for X
# instead of actual training data.
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

mean_scores = []
for kind in kinds:
    svc = LinearSVC(random_state=0)
    cv_scores = cross_val_score(svc,
                                connectivity_biomarkers[kind],
                                y=adhd_labels,
                                cv=cv,
                                groups=adhd_labels,
                                scoring='accuracy',
                                )
    mean_scores.append(cv_scores.mean())

Finally, we can display the classification scores.



In [ ]:
from nilearn.plotting import show

plt.figure(figsize=(6, 4))
positions = np.arange(len(kinds)) * .1 + .1
plt.barh(positions, mean_scores, align='center', height=.05)
yticks = [kind.replace(' ', '\n') for kind in kinds]
plt.yticks(positions, yticks)
plt.xlabel('Classification accuracy')
plt.grid(True)
plt.tight_layout()

show()